In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt


In [2]:
torch.__version__

'2.1.0+cu121'

In [3]:
torchvision.__version__

'0.16.0+cu121'

# Loading the data

In [4]:
ROOT_DIR = 'FashionMNIST'

In [5]:
train_data = torchvision.datasets.FashionMNIST(
    root= ROOT_DIR,
    train= True,
    download = True,
    transform= torchvision.transforms.ToTensor(),
)

test_data = torchvision.datasets.FashionMNIST(
    root= ROOT_DIR,
    train= False,
    download = True,
    transform= torchvision.transforms.ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11542457.52it/s]


Extracting FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 211201.81it/s]


Extracting FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3907902.53it/s]


Extracting FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20312584.19it/s]


Extracting FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



In [6]:
BATCH = 32
train_data_loder = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size= BATCH)
test_data_loder = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size= BATCH)

In [7]:
type(train_data_loder)

torch.utils.data.dataloader.DataLoader

In [8]:
for X, y in test_data_loder:
    print(X.shape)
    print(y.shape, "\nDtype",y.dtype, "\nunique",torch.unique(y))
    print(type(X))
    break

torch.Size([32, 1, 28, 28])
torch.Size([32]) 
Dtype torch.int64 
unique tensor([0, 1, 3, 4, 5, 6, 7, 8, 9])
<class 'torch.Tensor'>


In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Building Network

In [20]:
class Network(torch.nn.Module):
    def __init__(self,):
        super(Network, self).__init__()

        self.flatten = torch.nn.Flatten()

        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(28*28, 512),         # similar as dense layer
            torch.nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 10)
        )


    def forward(self, X):
        X = self.flatten(X)
        logits = self.linear_relu_stack(X)

        return logits

In [21]:
model = Network()
print(model)

Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [22]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-4)

In [23]:
def train(data_loder, model, loss_fn, optimizer):
    if torch.cuda.is_available():
        model.to('cuda')

    size = len(data_loder.dataset)

    for batch, (X, y) in enumerate(data_loder):
        # register to device
        X = X.to(device)
        y = y.to(device)

        # error calculation
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagration
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch % 100 == 0:
            LOSS, CURRENT = loss.item(), batch * len(X)
            print(f"\t\tTraining Loss: \t{LOSS}, [{CURRENT}/{size}]")

In [24]:
def test(data_loder, model):
    if torch.cuda.is_available():
        model.to('cuda')

    size = len(data_loder.dataset)
    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in data_loder:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss = test_loss + loss_fn(pred, y).item()
            correct = correct + (pred.argmax(1)==y).type(torch.float).sum().item()


    test_loss = test_loss/size
    correct = correct/size
    print(f"\t\tTest loss: \t {100*correct}, Avarage loss: \t{test_loss}")

In [25]:
EPOCHS = 5

for epoch in range(EPOCHS):
    print(f"------------------Epoch: {epoch+1}-------------------")
    train(train_data_loder, model, loss_fn, optimizer)
    test(test_data_loder, model)


print("........................Done training..................")

------------------Epoch: 1-------------------
		Training Loss: 	2.2957253456115723, [0/60000]
		Training Loss: 	2.2908387184143066, [3200/60000]
		Training Loss: 	2.3011457920074463, [6400/60000]
		Training Loss: 	2.3007149696350098, [9600/60000]
		Training Loss: 	2.2894747257232666, [12800/60000]
		Training Loss: 	2.2965781688690186, [16000/60000]
		Training Loss: 	2.2979094982147217, [19200/60000]
		Training Loss: 	2.304590940475464, [22400/60000]
		Training Loss: 	2.2982828617095947, [25600/60000]
		Training Loss: 	2.3149101734161377, [28800/60000]
		Training Loss: 	2.303863763809204, [32000/60000]
		Training Loss: 	2.307154655456543, [35200/60000]
		Training Loss: 	2.2914066314697266, [38400/60000]
		Training Loss: 	2.2915520668029785, [41600/60000]
		Training Loss: 	2.306442975997925, [44800/60000]
		Training Loss: 	2.2897536754608154, [48000/60000]
		Training Loss: 	2.3016276359558105, [51200/60000]
		Training Loss: 	2.2948148250579834, [54400/60000]
		Training Loss: 	2.313228845

In [26]:
torch.save(model, "model.pth")

In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]


model.eval()

X, y = torch.tensor(test_data[0][0]).to(device), torch.tensor(test_data[0][1]).to(device)

with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print("Predicted : ",predicted, "Actual",actual)

Predicted :  Ankle boot Actual Ankle boot


<ipython-input-27-8994e7962d4d>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X, y = torch.tensor(test_data[0][0]).to(device), torch.tensor(test_data[0][1]).to(device)
